In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train_data.csv')
y = df['poi']
df.drop(['name', 'email_address', 'poi'], axis=1, inplace=True)
df.drop(['loan_advances'], axis=1, inplace=True)
print(df.shape)
df.head(5)

(113, 18)


,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,1624396.0,22884.0,39.0,13.0,14.0,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,NaN,118134.0,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,NaN,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [3]:
fin_features = ['salary', 'deferral_payments', 'total_payments', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'director_fees',
                'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock']
email_features = ['to_messages', 'from_poi_to_this_person', 'from_messages', 
                  'from_this_person_to_poi', 'shared_receipt_with_poi']

In [4]:
y.value_counts()

False    100
True      13
Name: poi, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bonus                      61 non-null     float64
 1   deferral_payments          28 non-null     float64
 2   deferred_income            34 non-null     float64
 3   director_fees              13 non-null     float64
 4   exercised_stock_options    81 non-null     float64
 5   expenses                   73 non-null     float64
 6   from_messages              65 non-null     float64
 7   from_poi_to_this_person    65 non-null     float64
 8   from_this_person_to_poi    65 non-null     float64
 9   long_term_incentive        49 non-null     float64
 10  other                      69 non-null     float64
 11  restricted_stock           82 non-null     float64
 12  restricted_stock_deferred  10 non-null     float64
 13  salary                     73 non-null     float64

In [6]:
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    
    return missing_data
    
miss_ratio_df = na_check(df)
miss_ratio_df

,Missing Ratio
restricted_stock_deferred,91.150442
director_fees,88.495575
deferral_payments,75.221239
deferred_income,69.911504
long_term_incentive,56.637168
bonus,46.017699
from_messages,42.477876
from_poi_to_this_person,42.477876
from_this_person_to_poi,42.477876
shared_receipt_with_poi,42.477876


In [7]:
overmiss_cols = list(miss_ratio_df[miss_ratio_df['Missing Ratio'] > 60].index)
df = df.drop(overmiss_cols, axis=1)
df

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,1624396.0,22884.0,39.0,13.0,14.0,NaN,1573324.0,869220.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,118134.0,NaN,NaN,NaN,602671.0,907502.0,985032.0,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,2188.0,2598.0,1639297.0,126027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,500000.0,NaN,16514.0,NaN,NaN,NaN,NaN,356071.0,511734.0,239502.0,NaN,NaN,1112087.0,511734.0
109,NaN,NaN,49288.0,41.0,13.0,1.0,NaN,NaN,NaN,NaN,761.0,1486.0,49288.0,NaN
110,300000.0,28798.0,31653.0,67.0,7.0,0.0,NaN,NaN,NaN,655037.0,728.0,898.0,1386690.0,28798.0
111,NaN,NaN,475.0,19.0,4.0,7.0,NaN,NaN,2217299.0,NaN,471.0,573.0,475.0,2217299.0


In [8]:
fin_ratio_cols = ['bonus', 'expenses', 'other', 'salary']

for col in fin_ratio_cols:
    df[col + '_ratio'] = df[col] / df['total_payments']

df['to_poi_ratio'] = df['from_poi_to_this_person'] / df['to_messages']
df['from_poi_ratio'] = df['from_this_person_to_poi'] / df['from_messages']
df['stock_to_salary'] = df['total_stock_value'] / df['salary']
# df2 = df.copy()
df

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,...,to_messages,total_payments,total_stock_value,bonus_ratio,expenses_ratio,other_ratio,salary_ratio,to_poi_ratio,from_poi_ratio,stock_to_salary
0,1750000.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,...,905.0,505050.0,22542539.0,3.465003,0.092961,0.346182,0.832860,0.046409,0.222222,53.591559
1,5600000.0,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,...,3627.0,8682716.0,26093672.0,0.644959,0.003379,0.002548,0.127985,0.024262,0.277778,23.481201
2,200000.0,1624396.0,22884.0,39.0,13.0,14.0,NaN,1573324.0,869220.0,211844.0,...,225.0,2003885.0,2493616.0,0.099806,0.011420,0.785137,0.105717,0.057778,0.358974,11.771001
3,800000.0,NaN,118134.0,NaN,NaN,NaN,602671.0,907502.0,985032.0,224305.0,...,NaN,2652612.0,985032.0,0.301590,0.044535,0.342116,0.084560,NaN,NaN,4.391485
4,1250000.0,NaN,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,...,2598.0,1639297.0,126027.0,0.762522,0.021850,0.000296,0.146520,0.076597,0.173611,0.524699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,500000.0,NaN,16514.0,NaN,NaN,NaN,NaN,356071.0,511734.0,239502.0,...,NaN,1112087.0,511734.0,0.449605,0.014850,0.320183,0.215363,NaN,NaN,2.136659
109,NaN,NaN,49288.0,41.0,13.0,1.0,NaN,NaN,NaN,NaN,...,1486.0,49288.0,NaN,NaN,1.000000,NaN,NaN,0.008748,0.024390,NaN
110,300000.0,28798.0,31653.0,67.0,7.0,0.0,NaN,NaN,NaN,655037.0,...,898.0,1386690.0,28798.0,0.216343,0.022826,NaN,0.472375,0.007795,0.000000,0.043964
111,NaN,NaN,475.0,19.0,4.0,7.0,NaN,NaN,2217299.0,NaN,...,573.0,475.0,2217299.0,NaN,1.000000,NaN,NaN,0.006981,0.368421,NaN


In [9]:
df.fillna(0, inplace=True)
df.to_csv('丟棄過低+全補0不動.csv')

In [10]:
df

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,long_term_incentive,other,restricted_stock,salary,...,to_messages,total_payments,total_stock_value,bonus_ratio,expenses_ratio,other_ratio,salary_ratio,to_poi_ratio,from_poi_ratio,stock_to_salary
0,1750000.0,19794175.0,46950.0,18.0,42.0,4.0,1617011.0,174839.0,2748364.0,420636.0,...,905.0,505050.0,22542539.0,3.465003,0.092961,0.346182,0.832860,0.046409,0.222222,53.591559
1,5600000.0,19250000.0,29336.0,108.0,88.0,30.0,1920000.0,22122.0,6843672.0,1111258.0,...,3627.0,8682716.0,26093672.0,0.644959,0.003379,0.002548,0.127985,0.024262,0.277778,23.481201
2,200000.0,1624396.0,22884.0,39.0,13.0,14.0,0.0,1573324.0,869220.0,211844.0,...,225.0,2003885.0,2493616.0,0.099806,0.011420,0.785137,0.105717,0.057778,0.358974,11.771001
3,800000.0,0.0,118134.0,0.0,0.0,0.0,602671.0,907502.0,985032.0,224305.0,...,0.0,2652612.0,985032.0,0.301590,0.044535,0.342116,0.084560,0.000000,0.000000,4.391485
4,1250000.0,0.0,35818.0,144.0,199.0,25.0,375304.0,486.0,126027.0,240189.0,...,2598.0,1639297.0,126027.0,0.762522,0.021850,0.000296,0.146520,0.076597,0.173611,0.524699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,500000.0,0.0,16514.0,0.0,0.0,0.0,0.0,356071.0,511734.0,239502.0,...,0.0,1112087.0,511734.0,0.449605,0.014850,0.320183,0.215363,0.000000,0.000000,2.136659
109,0.0,0.0,49288.0,41.0,13.0,1.0,0.0,0.0,0.0,0.0,...,1486.0,49288.0,0.0,0.000000,1.000000,0.000000,0.000000,0.008748,0.024390,0.000000
110,300000.0,28798.0,31653.0,67.0,7.0,0.0,0.0,0.0,0.0,655037.0,...,898.0,1386690.0,28798.0,0.216343,0.022826,0.000000,0.472375,0.007795,0.000000,0.043964
111,0.0,0.0,475.0,19.0,4.0,7.0,0.0,0.0,2217299.0,0.0,...,573.0,475.0,2217299.0,0.000000,1.000000,0.000000,0.000000,0.006981,0.368421,0.000000
